# ImageCAS dataset EDA

In [3]:
import autorootcwd

## 1. Check data

In [35]:
import os
import nibabel as nib
import pandas as pd
from tqdm import tqdm

In [36]:
base_path = "data/imageCAS"

Check data location and Count data

In [37]:
def check_missing_files(base_path):
    directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    total_dirs = len(directories)
    
    print(f"\nTotal patient directory count: {total_dirs}")

    missing_patients = []
    
    for dir_name in tqdm(directories, desc="Checking missing files"):
        dir_path = os.path.join(base_path, dir_name)
        img_exists = os.path.exists(os.path.join(dir_path, 'img.nii.gz'))
        label_exists = os.path.exists(os.path.join(dir_path, 'label.nii.gz'))
        
        if not (img_exists and label_exists):
            missing_files = []
            if not img_exists: missing_files.append('img.nii.gz')
            if not label_exists: missing_files.append('label.nii.gz')
            missing_patients.append((dir_name, missing_files))
    
    if missing_patients:
        print("\nPatients with missing files:")
        for patient_id, missing_files in missing_patients:
            print(f"Patient {patient_id}: {', '.join(missing_files)} missing")
    else:
        print("\nAll patient data have img.nii.gz and label.nii.gz files.")

check_missing_files(base_path)


Total patient directory count: 1000


Checking missing files: 100%|██████████| 1000/1000 [00:00<00:00, 21739.24it/s]


All patient data have img.nii.gz and label.nii.gz files.


Get nifTi file metadata

In [42]:
data_number = 1
nii = nib.load("data/imageCAS/{}/img.nii.gz".format(data_number))

output_dir = "src/eda/"
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, "imageCAS_nifti_header_info.txt")

with open(output_file, 'w') as f:

    f.write("=== NIfTI Header Info [{}]===\n\n".format(data_number))
    
    f.write(str(nii.header))
    f.write("\n\n=== Detailed Header Info ===\n\n")
    
    for field in nii.header:
        f.write(f"{field}: {nii.header[field]}\n")
    
    f.write("\n=== Additional Info ===\n")
    f.write(f"data type: {nii.get_data_dtype()}\n")
    f.write(f"image size: {nii.shape}\n")
    f.write(f"header size: {nii.header.sizeof_hdr} bytes\n")

print(f"header info is saved in {output_file}")

header info is saved in src/eda/imageCAS_nifti_header_info.txt


Check shape and voxel size of the image and label match

In [34]:
def check_match_img_and_label(base_path):
    results = []
    
    for i in tqdm(range(1, 1001)):
        folder = str(i)
        folder_path = os.path.join(base_path, folder)
        
        if not os.path.exists(folder_path):
            continue
            
        try:
            img_path = os.path.join(folder_path, 'img.nii.gz')
            label_path = os.path.join(folder_path, 'label.nii.gz')
            
            # image file analysis
            if os.path.exists(img_path):
                img_nii = nib.load(img_path)
                img_shape = img_nii.shape
                img_voxel = img_nii.header.get_zooms()
            else:
                img_shape = None
                img_voxel = None
                
            # label file analysis
            if os.path.exists(label_path):
                label_nii = nib.load(label_path)
                label_shape = label_nii.shape
                label_voxel = label_nii.header.get_zooms()
            else:
                label_shape = None
                label_voxel = None
                
            results.append({
                'folder': i,
                'img_shape': img_shape,
                'img_voxel_size': img_voxel,
                'label_shape': label_shape,
                'label_voxel_size': label_voxel
            })
            
        except Exception as e:
            print(f"[Error] Patient id {i}: {str(e)}")
            
    return pd.DataFrame(results)

In [43]:
df = check_match_img_and_label(base_path)

df.to_csv('src/eda/nifti_analysis_results.csv', index=False)

print("\n=== Analysis Results ===")
print("\nUnique image shape:")
print(df['img_shape'].value_counts())

print("\nUnique image voxel size:")
print(df['img_voxel_size'].value_counts())

mismatched_shape = df[df['img_shape'] != df['label_shape']]
if not mismatched_shape.empty:
    print("\nCases where image and label shape do not match:")
    print(mismatched_shape[['folder', 'img_shape', 'label_shape']])
else:
    print("\nAll image and label shapes match.")

mismatched_voxel = df[df['img_voxel_size'] != df['label_voxel_size']]
if not mismatched_voxel.empty:
    print("\nCases where image and label voxel size do not match:")
    print(mismatched_voxel[['folder', 'img_voxel_size', 'label_voxel_size']])
else:
    print("\nAll image and label voxel sizes match.")

# check missing files
missing = df[df['img_shape'].isna() | df['label_shape'].isna()]
if not missing.empty:
    print("\nFolders with missing files:")
    print(missing['folder'].tolist())

100%|██████████| 1000/1000 [00:01<00:00, 990.68it/s]


=== Analysis Results ===

Unique image shape:
img_shape
(512, 512, 275)    621
(512, 512, 206)    126
(512, 512, 245)     18
(512, 512, 259)     15
(512, 512, 241)     15
(512, 512, 229)     13
(512, 512, 223)     13
(512, 512, 249)     12
(512, 512, 243)     12
(512, 512, 239)     12
(512, 512, 247)     11
(512, 512, 261)     11
(512, 512, 221)     10
(512, 512, 253)     10
(512, 512, 231)     10
(512, 512, 233)      9
(512, 512, 225)      9
(512, 512, 257)      8
(512, 512, 227)      8
(512, 512, 237)      7
(512, 512, 255)      6
(512, 512, 219)      6
(512, 512, 263)      6
(512, 512, 235)      6
(512, 512, 251)      4
(512, 512, 217)      4
(512, 512, 209)      2
(512, 512, 248)      2
(512, 512, 267)      2
(512, 512, 228)      1
(512, 512, 215)      1
(512, 512, 187)      1
(512, 512, 240)      1
(512, 512, 213)      1
(512, 512, 236)      1
(512, 512, 166)      1
(512, 512, 269)      1
(512, 512, 277)      1
(512, 512, 265)      1
(512, 512, 268)      1
(512, 512, 200)      1


Check the voxel size and image shape for each data

In [33]:
def analyze_voxel_size_and_image_shape(base_path):
    results = []
    
    for i in tqdm(range(1, 1001)):
        folder = str(i)
        img_path = os.path.join(base_path, folder, 'img.nii.gz')
        
        if not os.path.exists(img_path):
            continue
            
        try:
            img_nii = nib.load(img_path)
            voxel_size = img_nii.header.get_zooms()
            shape = img_nii.shape
            
            results.append({
                'patient_id': i,
                'voxel_x': voxel_size[0],
                'voxel_y': voxel_size[1],
                'voxel_z': voxel_size[2],
                'shape_x': shape[0],
                'shape_y': shape[1],
                'shape_z': shape[2]
            })
            
        except Exception as e:
            print(f"[Error] Patient id {i}: {str(e)}")
    
    return pd.DataFrame(results)


In [45]:
df = analyze_voxel_size_and_image_shape(base_path)

print("\n=== Voxel Size Statistics ===")
stats = df[['voxel_x', 'voxel_y', 'voxel_z']].describe()
print(stats)

# unique voxel size
unique_voxels = df.groupby(['voxel_x', 'voxel_y', 'voxel_z']).size().reset_index(name='count')
unique_voxels = unique_voxels.sort_values('count', ascending=False)
print("\n=== Unique Voxel Size Combination ===")
print(unique_voxels)

df.to_csv('src/eda/voxel_size_comparison.csv', index=False)

# check outliers (cases more than 2 standard deviations from the mean)
print("\n=== Check Outliers ===")
for col in ['voxel_x', 'voxel_y', 'voxel_z']:
    mean = df[col].mean()
    std = df[col].std()
    outliers = df[abs(df[col] - mean) > 2*std]
    if not outliers.empty:
        print(f"\n{col} outliers:")
        print(outliers[['patient_id', col]])

100%|██████████| 1000/1000 [00:00<00:00, 1888.08it/s]


=== Voxel Size Statistics ===
           voxel_x      voxel_y  voxel_z
count  1000.000000  1000.000000   1000.0
mean      0.352982     0.352982      0.5
std       0.029897     0.029897      0.0
min       0.289062     0.289062      0.5
25%       0.320312     0.320312      0.5
50%       0.349609     0.349609      0.5
75%       0.371582     0.371582      0.5
max       0.464844     0.464844      0.5

=== Unique Voxel Size Combination ===
     voxel_x   voxel_y  voxel_z  count
6   0.318359  0.318359      0.5    240
20  0.345703  0.345703      0.5     35
28  0.361328  0.361328      0.5     33
18  0.341797  0.341797      0.5     29
19  0.343750  0.343750      0.5     29
..       ...       ...      ...    ...
59  0.421875  0.421875      0.5      1
67  0.445312  0.445312      0.5      1
66  0.435547  0.435547      0.5      1
70  0.453125  0.453125      0.5      1
71  0.458984  0.458984      0.5      1

[73 rows x 4 columns]

=== Check Outliers ===

voxel_x outliers:
     patient_id   voxel_x
1

In [46]:
df = analyze_voxel_size_and_image_shape(base_path)

print("\n=== Image Shape Statistics ===")
stats = df[['shape_x', 'shape_y', 'shape_z']].describe()
print(stats)

#unique image shape
unique_shapes = df.groupby(['shape_x', 'shape_y', 'shape_z']).size().reset_index(name='count')
unique_shapes = unique_shapes.sort_values('count', ascending=False)
print("\n=== Unique Image Shape Combination ===")
print(unique_shapes)

df.to_csv('src/eda/image_shape_comparison.csv', index=False)

# check outliers (cases more than 2 standard deviations from the mean)
print("\n=== Check Outliers ===")
for col in ['shape_x', 'shape_y', 'shape_z']:
    mean = df[col].mean()
    std = df[col].std()
    outliers = df[abs(df[col] - mean) > 2*std]
    if not outliers.empty:
        print(f"\n{col} outliers:")
        print(outliers[['patient_id', col]])

100%|██████████| 1000/1000 [00:00<00:00, 1924.91it/s]


=== Image Shape Statistics ===
       shape_x  shape_y      shape_z
count   1000.0   1000.0  1000.000000
mean     512.0    512.0   257.496000
std        0.0      0.0    25.678227
min      512.0    512.0   166.000000
25%      512.0    512.0   241.000000
50%      512.0    512.0   275.000000
75%      512.0    512.0   275.000000
max      512.0    512.0   277.000000

=== Unique Image Shape Combination ===
    shape_x  shape_y  shape_z  count
39      512      512      275    621
3       512      512      206    126
24      512      512      245     18
22      512      512      241     15
32      512      512      259     15
14      512      512      229     13
10      512      512      223     13
20      512      512      239     12
23      512      512      243     12
27      512      512      249     12
25      512      512      247     11
33      512      512      261     11
29      512      512      253     10
15      512      512      231     10
9       512      512      221     10
11 

## Data Preprocessing

In [1]:
import os
import gc
import nibabel as nib
import numpy as np
from scipy import ndimage
from tqdm import tqdm
from monai.transforms import Resize
import multiprocessing as mp
from functools import partial

In [2]:
import autorootcwd

base_path = "data/imageCAS"

1. Adjust voxel size and image shape

In [47]:
def preprocess_imageCAS(base_path, output_base="D:/imageCAS_preprocessed", target_spacing=(0.35, 0.35, 0.5), target_xy_size=512, check_interval=100):
    os.makedirs(output_base, exist_ok=True)
    
    directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    total_dirs = len(directories)
    
    print(f"\nTotal patient directory count: {total_dirs}")
    print(f"Target voxel spacing: {target_spacing}")
    print(f"Target x,y size: {target_xy_size}")
    print(f"Output directory: {output_base}")
    
    errors = []
    skipped = []
    processed = []
    
    def print_interim_results(current_count):
        print(f"\n=== Interim Results (Processed directories: {current_count}/{total_dirs}) ===")
        print(f"Processed: {len(processed)} files")
        print(f"Skipped: {len(skipped)} files")
        print(f"Errors: {len(errors)} files")
        if errors:
            print("\nRecent errors:")
            for error in errors[-5:]:
                print(error)
        print("="*50)

    def resample_image(image, current_spacing, target_spacing, affine, is_label=False):
        try:
            image = image.astype(np.float32)
            scale_factors = np.array(current_spacing) / np.array(target_spacing)
            
            order = 0 if is_label else 1
            resampled_image = ndimage.zoom(image, scale_factors, order=order)
            
            scaling_matrix = np.diag(1 / scale_factors)
            new_affine = affine.copy()
            new_affine[:3, :3] = affine[:3, :3] @ scaling_matrix[:3, :3]
            
            return resampled_image, new_affine
        except Exception as e:
            raise Exception(f"[Error] Resampling failed: {str(e)}")

    for idx, dir_name in enumerate(tqdm(directories, desc="Data preprocessing")):
            input_dir_path = os.path.join(base_path, dir_name)
            output_dir_path = os.path.join(output_base, dir_name)
            os.makedirs(output_dir_path, exist_ok=True)
            
            for file_name in ['img.nii.gz', 'label.nii.gz']:
                input_file_path = os.path.join(input_dir_path, file_name)
                output_file_path = os.path.join(output_dir_path, file_name)
                is_label = file_name == 'label.nii.gz'

                if not os.path.exists(input_file_path):
                    errors.append(f"Patient {dir_name}: {file_name} file missing")
                    continue
                
                try:
                    nii = nib.load(input_file_path)
                    current_spacing = nii.header.get_zooms()
                    current_shape = nii.shape
                    
                    def is_close_enough(current, target, rtol=1e-5):
                        return np.allclose(current, target, rtol=rtol)

                    needs_processing = (
                        not is_close_enough(current_spacing, target_spacing) or 
                        current_shape[0] != target_xy_size or 
                        current_shape[1] != target_xy_size
                    )

                    if not needs_processing:
                        skipped.append(f"{dir_name}/{file_name}")
                        continue

                    image_data = nii.get_fdata(dtype=np.float32)
                    
                    resampled_image, new_affine = resample_image(
                        image_data, current_spacing, target_spacing, nii.affine, is_label
                    )

                    resize_mode = 'nearest' if is_label else 'bilinear'
                    resize_transform = Resize(
                        spatial_size=(target_xy_size, target_xy_size, resampled_image.shape[2]),
                        mode=resize_mode
                    )
                    resized_image = resize_transform(resampled_image[None])[0]
                    resized_image = resized_image.numpy()

                    if is_label:
                        resized_image = np.round(resized_image).astype(np.int32)

                    new_nii = nib.Nifti1Image(resized_image, affine=new_affine)
                    new_nii.header.set_zooms(target_spacing)
                    nib.save(new_nii, output_file_path)

                    processed.append(f"{dir_name}/{file_name}")

                    del nii, image_data, resampled_image, resized_image, new_nii
                    gc.collect()

                except Exception as e:
                    errors.append(f"Patient {dir_name}, {file_name} processing error: {str(e)}")
                    gc.collect()

            if (idx + 1) % check_interval == 0:
                print_interim_results(idx + 1)

    print("\n=== Final Results ===")
    print(f"Processed file count: {len(processed)}")
    print(f"Skipped file count: {len(skipped)}")
    print(f"Error count: {len(errors)}")
    if errors:
        print("Error files: ", errors)
    
    if errors:
        print("\nAll occurred errors:")
        for error in errors:
            print(error)

In [ ]:
target_spacing = (0.35, 0.35, 0.5)
target_xy_size = 512
base_path = "data/imageCAS"
output_base = "D:/imageCAS_preprocessed"

preprocess_imageCAS(base_path, output_base, target_spacing, target_xy_size, check_interval=200)


Total patient directory count: 1000
Target voxel spacing: (0.35, 0.35, 0.5)
Target x,y size: 512
Output directory: D:/imageCAS_preprocessed_f


Data preprocessing:   0%|          | 0/1000 [00:00<?, ?it/s]

Data preprocessing:  22%|██▏       | 220/1000 [00:00<00:02, 299.96it/s]


=== Interim Results (Processed directories: 200/1000) ===
Processed: 0 files
Skipped: 400 files
Errors: 0 files


Data preprocessing:  42%|████▏     | 422/1000 [00:01<00:03, 188.01it/s]

...


=== Interim Results (Processed directories: 600/1000) ===
Processed: 0 files
Skipped: 1200 files
Errors: 0 files


Data preprocessing:  83%|████████▎ | 831/1000 [00:03<00:00, 188.32it/s]


=== Interim Results (Processed directories: 800/1000) ===
Processed: 0 files
Skipped: 1600 files
Errors: 0 files


Data preprocessing: 100%|██████████| 1000/1000 [00:04<00:00, 213.99it/s]


=== Interim Results (Processed directories: 1000/1000) ===
Processed: 0 files
Skipped: 2000 files
Errors: 0 files

=== Final Results ===
Processed file count: 0
Skipped file count: 2000
Error count: 0


Preprocessing voxel size, image shape result

In [43]:
df = analyze_voxel_size_and_image_shape(base_path)

print("\n=== Voxel Size Statistics ===")
stats = df[['voxel_x', 'voxel_y', 'voxel_z']].describe()
print(stats)

# unique voxel size
unique_voxels = df.groupby(['voxel_x', 'voxel_y', 'voxel_z']).size().reset_index(name='count')
unique_voxels = unique_voxels.sort_values('count', ascending=False)
print("\n=== Unique Voxel Size Combination ===")
print(unique_voxels)

df.to_csv('src/eda/voxel_size_comparison.csv', index=False)

# check outliers (cases more than 2 standard deviations from the mean)
if (unique_voxels.shape[0] > 1):
    print("\n=== Check Outliers ===")
    for col in ['voxel_x', 'voxel_y', 'voxel_z']:
        mean = df[col].mean()
        std = df[col].std()
    outliers = df[abs(df[col] - mean) > 2*std]
    if not outliers.empty:
        print(f"\n{col} outliers:")
        print(outliers[['patient_id', col]])

100%|██████████| 1000/1000 [00:00<00:00, 1900.64it/s]


=== Voxel Size Statistics ===
       voxel_x  voxel_y  voxel_z
count  1000.00  1000.00   1000.0
mean      0.35     0.35      0.5
std       0.00     0.00      0.0
min       0.35     0.35      0.5
25%       0.35     0.35      0.5
50%       0.35     0.35      0.5
75%       0.35     0.35      0.5
max       0.35     0.35      0.5

=== Unique Voxel Size Combination ===
   voxel_x  voxel_y  voxel_z  count
0     0.35     0.35      0.5   1000


In [44]:
df = analyze_voxel_size_and_image_shape(base_path)

print("\n=== Image Shape Statistics ===")
stats = df[['shape_x', 'shape_y', 'shape_z']].describe()
print(stats)

#unique image shape
unique_shapes = df.groupby(['shape_x', 'shape_y', 'shape_z']).size().reset_index(name='count')
unique_shapes = unique_shapes.sort_values('count', ascending=False)
print("\n=== Unique Image Shape Combination ===")
print(unique_shapes)

df.to_csv('src/eda/image_shape_comparison.csv', index=False)

# check outliers (cases more than 2 standard deviations from the mean)
print("\n=== Check Outliers ===")
for col in ['shape_x', 'shape_y', 'shape_z']:
    mean = df[col].mean()
    std = df[col].std()
    outliers = df[abs(df[col] - mean) > 2*std]
    if not outliers.empty:
        print(f"\n{col} outliers:")
        print(outliers[['patient_id', col]])

100%|██████████| 1000/1000 [00:00<00:00, 1938.85it/s]


=== Image Shape Statistics ===
       shape_x  shape_y      shape_z
count   1000.0   1000.0  1000.000000
mean     512.0    512.0   257.496000
std        0.0      0.0    25.678227
min      512.0    512.0   166.000000
25%      512.0    512.0   241.000000
50%      512.0    512.0   275.000000
75%      512.0    512.0   275.000000
max      512.0    512.0   277.000000

=== Unique Image Shape Combination ===
    shape_x  shape_y  shape_z  count
39      512      512      275    621
3       512      512      206    126
24      512      512      245     18
22      512      512      241     15
32      512      512      259     15
10      512      512      223     13
14      512      512      229     13
20      512      512      239     12
27      512      512      249     12
23      512      512      243     12
25      512      512      247     11
33      512      512      261     11
15      512      512      231     10
29      512      512      253     10
9       512      512      221     10
16 